<a href="https://colab.research.google.com/github/Sartify/Gemma-3-Notebooks/blob/main/scripts/text_inference_gemma_with_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook running on L4, but you can quantize and run it on T4.

In [1]:
# Install the required dependencies for using Hugging Face's Transformers library
# This specific version includes support for Google's Gemma 3 model
!pip install -q git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3

# Upgrade and install bitsandbytes, a library for efficient model loading and quantization
!pip install -U -q bitsandbytes

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 95.4 MB/s 

In [2]:
# Import necessary libraries

# Hugging Face's Transformers library for working with Gemma 3 model
from transformers import AutoProcessor, Gemma3ForConditionalGeneration

# PIL (Python Imaging Library) for handling image processing
from PIL import Image

# Requests library to fetch images from URLs
import requests

# OpenCV (cv2) for image processing and manipulation
import cv2

# NumPy for numerical computations and handling image data as arrays
import numpy as np

# PyTorch for tensor computations and model execution
import torch

In [3]:
# Define the model ID - you can change this to any available Gemma 3 model variant
model_id = "google/gemma-3-4b-it"

# Load the Gemma 3 model for conditional generation
# device_map="auto" automatically assigns the model to an available GPU or CPU
model = Gemma3ForConditionalGeneration.from_pretrained(
    model_id, device_map="auto"
).eval()  # Set the model to evaluation mode (inference mode)

# Load the corresponding processor for the selected model
# The processor is responsible for preprocessing text and images before feeding them into the model
processor = AutoProcessor.from_pretrained(model_id)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

In [4]:
# Download a video file from the specified URL using wget
!wget https://raw.githubusercontent.com/Sartify/Gemma-3-Notebooks/main/images/video1.mp4

--2025-03-12 14:55:40--  https://raw.githubusercontent.com/Sartify/Gemma-3-Notebooks/main/images/video1.mp4
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47657513 (45M) [application/octet-stream]
Saving to: ‘video1.mp4’

video1.mp4          100%[===================>]  45.45M   284MB/s    in 0.2s    

2025-03-12 14:55:41 (284 MB/s) - ‘video1.mp4’ saved [47657513/47657513]



In [5]:
import cv2
import numpy as np
from PIL import Image

def downsample_video(video_path):
    """
    Extracts 6 evenly spaced frames from a given video and converts them to RGB format.

    Parameters:
        video_path (str): Path to the video file.

    Returns:
        frames (list): A list of tuples where each tuple contains:
            - A PIL Image (RGB converted frame)
            - A timestamp (in seconds) corresponding to the extracted frame
    """
    # Open the video file
    vidcap = cv2.VideoCapture(video_path)

    # Get total number of frames in the video
    total_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Get the frames per second (fps) of the video
    fps = vidcap.get(cv2.CAP_PROP_FPS)

    # List to store extracted frames
    frames = []

    # Select 6 evenly spaced frame indices across the video
    frame_indices = np.linspace(0, total_frames - 1, 6, dtype=int)

    # Iterate through selected frame indices
    for i in frame_indices:
        # Move the video capture to the specified frame index
        vidcap.set(cv2.CAP_PROP_POS_FRAMES, i)

        # Read the frame
        success, image = vidcap.read()

        if success:
            # Convert frame from BGR (OpenCV format) to RGB (PIL format)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Convert NumPy array to PIL Image
            pil_image = Image.fromarray(image)

            # Calculate the timestamp of the frame (rounded to 2 decimal places)
            timestamp = round(i / fps, 2)

            # Append the PIL Image and timestamp as a tuple to the frames list
            frames.append((pil_image, timestamp))

    # Release the video capture object
    vidcap.release()

    return frames

# Call the function to extract frames from the video file
frames = downsample_video("/content/video1.mp4")
frames

[(<PIL.Image.Image image mode=RGB size=3840x2160>, 0.0),
 (<PIL.Image.Image image mode=RGB size=3840x2160>, 3.4),
 (<PIL.Image.Image image mode=RGB size=3840x2160>, 6.84),
 (<PIL.Image.Image image mode=RGB size=3840x2160>, 10.24),
 (<PIL.Image.Image image mode=RGB size=3840x2160>, 13.68),
 (<PIL.Image.Image image mode=RGB size=3840x2160>, 17.12)]

In [6]:
# Define the message prompt structure for the Gemma model
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are a helpful assistant."}]
        # System message to set the assistant's role and behavior
    },

    {
        "role": "user",
        "content": [
            {"type": "text", "text": f"What is happening in this video? Count number of people and summarize the events."}
            # User's prompt asking Gemma to analyze and summarize the events in a given video
        ]
    }
]

In [7]:
# Iterate through the extracted frames from the video
for frame in frames:
    image, timestamp = frame  # Extract image and its corresponding timestamp

    # Append a textual description indicating the frame's timestamp
    messages[1]["content"].append({"type": "text", "text": f"Frame {timestamp}:"})

    # Save the frame as an image file with the timestamp in its filename
    image.save(f"image_{timestamp}.png")

    # Append the saved image file as a reference in the message content
    messages[1]["content"].append({"type": "image", "url": f"image_{timestamp}.png"})


In [8]:
# Process the message prompt and prepare it for the model
inputs = processor.apply_chat_template(
    messages,
    add_generation_prompt=True,  # Adds a prompt to indicate that a response is expected
    tokenize=True,  # Tokenizes the input for the model
    return_dict=True,  # Returns the processed output as a dictionary
    return_tensors="pt"  # Converts the processed data into PyTorch tensors
).to(model.device)  # Moves the tensors to the appropriate device (CPU or GPU)

In [9]:
# Get the length of the input tokens to track where the model's generated output starts
input_len = inputs["input_ids"].shape[-1]

# Generate text output from the model
generation = model.generate(
    **inputs,  # Pass processed input to the model
    max_new_tokens=1024,  # Limit the response to 500 new tokens
    do_sample=False  # Disable sampling for deterministic output (greedy decoding)
)

# Extract only the newly generated tokens (excluding the input prompt)
generation = generation[0][input_len:]

# Decode the generated tokens into human-readable text
decoded = processor.decode(generation, skip_special_tokens=True)

# Print the final output
print(decoded)

Okay, let's break down what's happening in these images.

**Number of People:**

There are three people consistently visible throughout the video: a man, a young girl, and a small child.

**Summary of Events:**

The video shows a family – a man, a young girl, and a small child – walking down a long, dirt road. The road stretches out into a field of golden wheat or barley. The shots are taken from a high angle, giving a panoramic view of the landscape and the family. 

The family appears to be walking together, enjoying a leisurely stroll through the countryside. The lighting suggests it's either early morning or late afternoon/early evening, with a warm, golden glow. 

The consistent shot angle and the unchanging scene suggest this is a continuous, peaceful moment captured in time. 

Do you want me to focus on a specific detail or aspect of the video, such as the setting, the family's activity, or the overall mood?
